네이버 고민상담카페의 게시물을 크롤링하는 코드.

게시물 넘버링 기반으로 크롤링을 진행하여 csv 파일에 저장하고, 만일 글이 삭제되었을 경우 오류 메시지를 출력.

In [0]:
import urllib.request
import random
import time
from selenium import webdriver
from bs4 import BeautifulSoup 
import csv
 
driver = webdriver.Chrome(executable_path = "C:/Users/...../chromedriver.exe")
driver.implicitly_wait(3)

#각종 설정
id = 'XXXXX'
pw = 'XXXXX'
cafe_id = 10577860
MAX_POSTS = 477029

In [0]:
#네이버 로그인
driver.get("https://nid.naver.com/nidlogin.login")
driver.find_element_by_id("id").send_keys(id)
driver.find_element_by_id("pw").send_keys(pw)
driver.find_element_by_xpath('//*[@id="frmNIDLogin"]/fieldset/input').click()
time.sleep(15)

In [0]:
url_cafe_base = 'https://cafe.naver.com/ArticleRead.nhn?clubid=' + str(cafe_id) + '&page=1&boardtype=L&articleid='
url_cafe_extra = '&referrerAllArticles=true'

In [0]:
idx = 0

f = open('allPosts.csv', 'w', encoding='utf-8', newline='')
writer = csv.writer(f)
writer.writerow(['번호', '제목', '내용', '댓글'])

for i in range(MAX_POSTS, MAX_POSTS - 20, -1): # 20개 단위로 크롤링
    #open post which index is i
    try:
        url_cafe = url_cafe_base + str(i) + url_cafe_extra
        driver.get(url_cafe)
        driver.switch_to.frame('cafe_main')
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
    except :
        print(i, " 번째 게시글이 존재하지 않습니다!")
        continue

    #get title and content of this post
    title = soup.select_one('span[class="b m-tcol-c"]').string
    content_tags = soup.select('#tbody p')
    content = ''.join([tags.get_text() for tags in content_tags])

    #get all replies of this post as reply
    replies = []
    reply = ''
    tag_all = soup.body.find('ul', id='cmt_list')
    for k in tag_all:
       if k.find_all('span', class_='comm_body'):
            replies.append(k.div.find('span', class_="comm_body").get_text(strip=True))
    for i in replies:
        reply = reply + i + '\n'

    writer.writerow([str(idx), title, content, reply])
    idx += 1

f.close()